In [85]:
import numpy as np
from scipy.linalg import solve

A = np.array([[2.0, -1.0, 0.0, 0],
              [-1.0, 2.0, -1.0, 0],
              [0.0, -1.0, 2.0, -1],
              [0,0,-1,2]])
b = np.array([1.0, 0, 0,1.0])

**Método Iterativo de Jacobi**

In [86]:
def jacobi(A, b, x0 = None, e = 1e-5, max_i = 100):
    n = len(b)

    x = np.zeros_like(b) if x0 is None else x0

    for j in range(max_i):
        xi = np.copy(x)
        for i in range(n):
            sum_terms = np.dot(A[i, :i], x[:i]) + np.dot(A[i, i+1:], x[i+1:])
            xi[i] = (b[i] - sum_terms) / A[i, i]

        if np.linalg.norm(xi - x) < e: return xi, j+1
        x = xi
    return x, max_i

x, iters = jacobi(A.copy(), b.copy())

print(iters)
print(x)

51
[0.99998537 0.99997632 0.99997632 0.99998537]


**Método Iterativo de Gauss-Seidel**

In [87]:
def gauss_seidel(A, b, x0 = None, e = 1e-5, max_i = 100):
    n = len(b)
    x = np.zeros_like(b) if x0 is None else x0

    for j in range(max_i):
        xi = np.copy(x)

        for i in range(n):
            sum_terms = np.dot(A[i, :i], xi[:i]) + np.dot(A[i, i+1:], x[i+1:])
            xi[i] = (b[i] - sum_terms) / A[i, i]

        if np.linalg.norm(xi - x) < e: return xi, j+1
        x = xi
    return x, max_i

x, iters = gauss_seidel(A.copy(), b.copy())

print(iters)
print(x)

28
[0.9999919  0.9999894  0.99999142 0.99999571]


**Método Iterativo Sobre-Relaxação Sucessiva (SOR)**

In [88]:
def sor(A, b, x0 = None, omega = 1.0, e = 1e-5, max_i = 100):
    n = len(b)
    x = np.zeros_like(b) if x0 is None else x0

    for j in range(max_i):
        xi = np.copy(x)

        for i in range(n):
            sum_terms = np.dot(A[i, :i], xi[:i]) + np.dot(A[i, i+1:], x[i+1:])
            xi[i] = (1 - omega) * x[i] + omega * (b[i] - sum_terms) / A[i, i]

        if np.linalg.norm(xi - x) < e: return xi, j+1 
        x = xi
    return x, max_i
omega = 1.27
x1, iters = sor(A.copy(), b.copy(), omega = omega)

print(iters)
print(x1)

11
[0.99999849 1.00000104 1.00000073 1.00000027]


**Método do Refinamento Iterativo**

In [89]:
def iterative_refinement(A, b, x0, e = 1e-5, max_i = 10):
    x = x0
    print(x0)
    
    for j in range(max_i):
        r = b - np.dot(A, x)
        xi = solve(A.copy(),r)
        x += xi
        
        if np.linalg.norm(xi) < e: return x, j+1 
    return x, max_i

x0 = x
x, iters = iterative_refinement(A.copy(), b.copy(), x0)

print(iters)
print(x)


[0.9999919  0.9999894  0.99999142 0.99999571]
2
[1. 1. 1. 1.]


**Gradiente Conjugado Precondicionado**

In [90]:
def preconditioned_conjugated_gradient(A, b, M = None, x0 = None, e = 1e-5, max_i = 10):
    n = len(b)
    
    x = np.zeros(n) if x0 is None else x0
    r = b - np.dot(A, x)

    w = r if M is None else np.linalg.solve(M, r)
    v = w.copy() if M is None else np.linalg.solve(M, w)
    
    alpha = np.dot(w, w)
    
    for j in range(max_i):
        if np.linalg.norm(v) < e: break
        
        u = np.dot(A, v)
        t = alpha / np.dot(v, u)

        x += t * v
        r -= t * u

        w = r if M is None else np.linalg.solve(M, r)
        
        beta = np.dot(w, w)
        s = beta / alpha
        alpha = beta

        v = w + s * v    

        if np.linalg.norm(r) < e: break
    return x, j + 1

A = np.array([[4,3,0.0],
              [3,4,-1],
              [0,-1,4]])
b = np.array([24,30,-24])

x, iters = preconditioned_conjugated_gradient(A, b)

print(iters)
print(x)

3
[ 3.  4. -5.]
